In [ ]:
import pandas as pd
from tqdm import tqdm
from pykrx import stock

# 데이터 불러오기
pricedata = pd.read_csv('enlist_ver_2.0.csv')
fsdata = pd.read_csv('kor_fs.csv', encoding='cp949')

# 필요한 컬럼명 수정 및 인덱스 초기화
fsdata.columns = ['회사명', '종목코드', '날짜', ... , '유동자산']  # 컬럼명 재설정
fsdata.reset_index(drop=True, inplace=True)
fsdata.to_csv('fs_ver_1.0.csv')

# 필요한 조건에 따라 데이터 필터링
for i in tqdm(uni):
    if len(fsdata[fsdata['종목코드']==i]['날짜'].unique()) != 12:
        fsdata = fsdata.drop(fsdata[fsdata['종목코드']==i].index)

# 주어진 코드 리스트에 따라 데이터 추출
pricelist = pricedata['종목코드'].unique().tolist()
fsdata_filt = pd.DataFrame()
for i in tqdm(pricelist):
   fsdata_filt = pd.concat([fsdata_filt, fsdata[fsdata['종목코드']==i]])

# 날짜 기준 데이터 필터링
date_list = ['2011/03', ... , '2019/06']
fsdata_filt_date = fsdata_filt.copy()
for i in tqdm(date_list):
    fsdata_filt_date = fsdata_filt_date.drop(fsdata_filt_date[fsdata_filt['날짜']==i].index)

# 추가 계산 및 지표 생성
fsdata_cal['ROA'] = fsdata_cal['당기순이익'] / fsdata_cal['자산']
... # 여러 가지 지표 생성

# 시가총액 데이터 추가
result = pd.DataFrame()
for date in tqdm(date_list):
    date_data = stock.get_market_cap_by_ticker(date)
    ...
fs_cap_data['PER'] = fs_cap_data['시가총액'] / fs_cap_data['당기순이익']

# 가격 데이터 전처리
pricedata = pd.read_csv('price_ver_3.0.csv')
pricedata.columns = ['X', 'Date', 'Open','High', 'Low', 'Close', 'Volume', '종목코드']
pricedata.drop('X', axis=1, inplace=True)
pricedata['Date'] = pd.to_datetime(pricedata['Date'])
pricedata = pricedata.groupby('종목코드').apply(lambda group: group.set_index('Date').sort_index()).reset_index(level=0, drop=True)
